In [3]:
import glob
import re

# taking stanford-parsed texts as tokens
# why? because LinkAppend requires tokenised sentences and we want the results to be comparable to other models
# plus, the parser is quite good
basepath = '/home/xilini/coref/multimodal_coref/data/nikolai_test_out/randomsample_20/stanford-det/'


In [2]:
def extract_sentences_from_text(text):
    # regex specific to stanford output format
    sentence_pattern = re.compile(r'Tokens:\n(.*?)\n\nConstituency parse:', re.DOTALL)
    token_pattern = re.compile(r'Text=([^\s\[]+)', re.DOTALL)

    matches = sentence_pattern.findall(text)
    reconstructed_sentences = []
    for tokens_text in matches:
        matches = token_pattern.findall(tokens_text)
        sentence = ' '.join(matches)
        reconstructed_sentences.append(sentence)
    return reconstructed_sentences

In [7]:
parsed_files = glob.glob(basepath + '*.out')
parsed_files = [f for f in parsed_files if 'checkpoint' not in f]

In [8]:
parsed_files

['/home/xilini/coref/multimodal_coref/data/nikolai_test_out/randomsample_20/stanford-det/doc_3780.out',
 '/home/xilini/coref/multimodal_coref/data/nikolai_test_out/randomsample_20/stanford-det/doc_12405.out',
 '/home/xilini/coref/multimodal_coref/data/nikolai_test_out/randomsample_20/stanford-det/doc_8217.out',
 '/home/xilini/coref/multimodal_coref/data/nikolai_test_out/randomsample_20/stanford-det/doc_12403.out',
 '/home/xilini/coref/multimodal_coref/data/nikolai_test_out/randomsample_20/stanford-det/doc_6377.out',
 '/home/xilini/coref/multimodal_coref/data/nikolai_test_out/randomsample_20/stanford-det/doc_5987.out',
 '/home/xilini/coref/multimodal_coref/data/nikolai_test_out/randomsample_20/stanford-det/doc_8135.out',
 '/home/xilini/coref/multimodal_coref/data/nikolai_test_out/randomsample_20/stanford-det/doc_4689.out',
 '/home/xilini/coref/multimodal_coref/data/nikolai_test_out/randomsample_20/stanford-det/doc_4842.out',
 '/home/xilini/coref/multimodal_coref/data/nikolai_test_out/ra

In [15]:
import tqdm

vwp_linkappend_texts = []

for file in tqdm.tqdm(parsed_files):

    with open(file, 'r') as f:
        content = f.read()
    sentences = extract_sentences_from_text(content)

    processed_data = []
    for i, sentence in enumerate(sentences, 1):
        words = sentence.split()
        tokens = [{'id': idx+1, 'text': word} for idx, word in enumerate(words)]
        processed_data.append({
            'id': i,
            'speaker': None,
            'text': sentence,
            'tokens': tokens
        })

    text_dict = {}
    text_dict['id'] = 'doc_' + str(file.split('_')[-1].split('.out')[0])
    text_dict['sentences'] = processed_data
    text_dict['coref_chains'] = None
    
    vwp_linkappend_texts.append(text_dict)

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 8242.71it/s]

doc_3780
doc_12405
doc_8217
doc_12403
doc_6377
doc_5987
doc_8135
doc_4689
doc_4842
doc_262
doc_317
doc_1829
doc_1374
doc_8676
doc_5262
doc_5649
doc_4533
doc_7596
doc_10650
doc_12679


In [17]:
import json
with open('/home/xilini/coref/multimodal_coref/data/nikolai_test_out/randomsample_20/link-append/inputs_random20.json', 'w') as f:
    json.dump(vwp_linkappend_texts, f)